In [29]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import splinter
import requests
import json
import pymongo

In [30]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = splinter.Browser('chrome', **executable_path)

# NASA Mars News

In [31]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

In [32]:
soup = bs(html, "html.parser")
# print(soup.prettify())

In [33]:
content = soup.find("div", class_ = 'content_page')

In [34]:
titles = content.find_all("div", class_= 'content_title')
news_title=titles[0].text.strip()
news_title

"A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes"

In [35]:
article_text = content.find_all("div", class_ = "article_teaser_body")
news_p= article_text[0].text
news_p

"Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. "

# JPL Mars Space Images - Featured Image

In [36]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html

In [37]:
browser.find_by_id("full_image").click()

In [38]:
browser.find_by_text("more info     ").click()

In [39]:
soup = bs(browser.html, "html.parser")
featured_img = soup.find("img", class_="main_image")["src"]
featured_img

'/spaceimages/images/largesize/PIA17563_hires.jpg'

In [40]:
former=url.split('?')[0]
former

'https://www.jpl.nasa.gov/spaceimages/'

In [41]:
latter=featured_img.split('/spaceimages/')[1]
latter

'images/largesize/PIA17563_hires.jpg'

In [42]:
featured_image_url= former+latter
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17563_hires.jpg'

# Mars Facts

In [43]:
url = 'https://space-facts.com/mars/'
# Extract tables
dfs = pd.read_html(url)
print(len(dfs))

3


In [44]:
mars_df= dfs[0]
mars_df.columns=["Description", "Mars"]
mars_df.set_index('Description')

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [45]:
planet_facts_table_html = mars_df.to_html()
print(planet_facts_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Mars</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
     

# Mars Hemispheres

In [46]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html

In [47]:
hemisphere_image_urls=[]

links = browser.find_by_css("a.product-item h3")
for i in range(len(links)):
    hemispheres={}
    browser.find_by_css("a.product-item h3")[i].click()
    soup = bs(browser.html, "html.parser")
    title= soup.find("h2", class_="title").text
    link=soup.find('div',class_='downloads')
    img_url=link.find("a")['href']
    
    hemispheres["title"]=title
    hemispheres["img_url"]=img_url

    hemisphere_image_urls.append(hemispheres)
    browser.back()

In [48]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# MongoDB

In [49]:
conn='mongodb://localhost:27017'
client= pymongo.MongoClient(conn)

In [50]:
db= client.mars_db

In [51]:
mars=db.mars

In [52]:
mars_data={'news_title': news_title,
        'news_p': news_p,
        'featured_image_url': featured_image_url,
        'planet_facts_table_html': planet_facts_table_html,
        'hemisphere_image_urls' : hemisphere_image_urls}
mars.insert_one(mars_data)